In [ ]:
import pygame
import math

# Определение цветов
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
COLORS = {
    "Red": (255, 0, 0),
    "Green": (0, 255, 0),
    "Blue": (0, 0, 255),
    "Yellow": (255, 255, 0)
}

SHAPES = {
    "Line": "line",
    "Rect": "rectangle",
    "Circle": "circle",
    "Square": "square",
    "Right_tr": "right_triangle",
    "Eq_tr": "equilateral_triangle",
    "Rhombus": "rhombus",
    "Eraser": "eraser"
}

BUTTON_WIDTH = 80
BUTTON_HEIGHT = 30
MENU_HEIGHT = 70
line_radius = 5      # Толщина линии
eraser_radius = 20   # Размер ластика

def draw_menu(screen, font, selected_color_name, selected_shape_name):
    # Отрисовка цветных кнопок
    x = 10
    y = 10
    for name, color in COLORS.items():
        rect = pygame.Rect(x, y, BUTTON_WIDTH, BUTTON_HEIGHT)
        pygame.draw.rect(screen, color, rect)
        if name == selected_color_name:
            pygame.draw.rect(screen, WHITE, rect, 3)
        else:
            pygame.draw.rect(screen, BLACK, rect, 2)
        text = font.render(name, True, BLACK)
        screen.blit(text, (x + 5, y + 5))
        x += BUTTON_WIDTH + 10

    # Отрисовка кнопок для фигур
    x = 10
    y = 40
    for name in SHAPES:
        rect = pygame.Rect(x, y, BUTTON_WIDTH, BUTTON_HEIGHT)
        pygame.draw.rect(screen, (200, 200, 200), rect)
        if SHAPES[name] == selected_shape_name:
            pygame.draw.rect(screen, (0, 0, 0), rect, 3)
        else:
            pygame.draw.rect(screen, BLACK, rect, 2)
        text = font.render(name, True, BLACK)
        screen.blit(text, (x + 2, y + 5))
        x += BUTTON_WIDTH + 10


def get_button_clicked(pos):
    x, y = pos
    if y < 40:
        index = (x - 10) // (BUTTON_WIDTH + 10)
        if 0 <= index < len(COLORS):
            return "color", list(COLORS.keys())[index]
    elif y < MENU_HEIGHT:
        index = (x - 10) // (BUTTON_WIDTH + 10)
        if 0 <= index < len(SHAPES):
            return "shape", list(SHAPES.keys())[index]
    return None, None


def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    pygame.display.set_caption("Меню рисования")
    font = pygame.font.SysFont(None, 20)
    clock = pygame.time.Clock()

    radius = 2
    selected_color_name = "Blue"
    selected_shape_name = "line"
    color = COLORS[selected_color_name]
    shape = selected_shape_name
    drawing = False
    start_pos = (0, 0)
    background_color = BLACK

    screen.fill(background_color)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return

            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    if event.pos[1] < MENU_HEIGHT:
                        type_, name = get_button_clicked(event.pos)
                        if type_ == "color":
                            selected_color_name = name
                            color = COLORS[name]
                        elif type_ == "shape":
                            selected_shape_name = SHAPES[name]
                            shape = selected_shape_name
                    else:
                        drawing = True
                        start_pos = event.pos
                        if shape == "line":
                            pygame.draw.circle(screen, color, event.pos, line_radius)

            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:
                    if drawing:
                        drawing = False
                        end_pos = event.pos
                        dx = end_pos[0] - start_pos[0]
                        dy = end_pos[1] - start_pos[1]

                        if shape == "rectangle":
                            rect = pygame.Rect(start_pos, (dx, dy))
                            pygame.draw.rect(screen, color, rect, line_radius)

                        elif shape == "circle":
                            circle_radius = int(math.hypot(dx, dy))
                            pygame.draw.circle(screen, color, start_pos, circle_radius, line_radius)

                        elif shape == "square":
                            side = max(abs(dx), abs(dy))
                            rect = pygame.Rect(start_pos[0], start_pos[1], side, side)
                            pygame.draw.rect(screen, color, rect, line_radius)

                        elif shape == "right_triangle":
                            p1 = start_pos
                            p2 = (start_pos[0], end_pos[1])
                            p3 = end_pos
                            pygame.draw.polygon(screen, color, [p1, p2, p3], line_radius)

                        elif shape == "equilateral_triangle":
                            side = max(abs(dx), abs(dy))
                            height = int((math.sqrt(3) / 2) * side)
                            p1 = start_pos
                            p2 = (start_pos[0] + side, start_pos[1])
                            p3 = (start_pos[0] + side // 2, start_pos[1] - height)
                            pygame.draw.polygon(screen, color, [p1, p2, p3], line_radius)

                        elif shape == "rhombus":
                            center_x = (start_pos[0] + end_pos[0]) // 2
                            center_y = (start_pos[1] + end_pos[1]) // 2
                            dx_half = abs(end_pos[0] - start_pos[0]) // 2
                            dy_half = abs(end_pos[1] - start_pos[1]) // 2
                            p1 = (center_x, center_y - dy_half)
                            p2 = (center_x + dx_half, center_y)
                            p3 = (center_x, center_y + dy_half)
                            p4 = (center_x - dx_half, center_y)
                            pygame.draw.polygon(screen, color, [p1, p2, p3, p4], line_radius)

            elif event.type == pygame.MOUSEMOTION:
                if drawing:
                    if shape == "line":
                        pygame.draw.circle(screen, color, event.pos, line_radius)
                    elif shape == "eraser":
                        pygame.draw.circle(screen, background_color, event.pos, eraser_radius)
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    screen.fill(background_color)

        draw_menu(screen, font, selected_color_name, selected_shape_name)
        pygame.display.flip()
        clock.tick(60)


if __name__ == "__main__":
    main()